In [ ]:
!pip install google-cloud-aiplatform --upgrade --user --quiet
!pip install langchain==0.0.353 chromadb==0.3.26 pydantic==1.10.8 typing-inspect==0.8.0 typing_extensions==4.5.0 pandas datasets google-api-python-client pypdf faiss-cpu transformers config --upgrade --user --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.1/803.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.3 MB/s eta 0:00:00
  Installing build dependen

In [ ]:
# Below libraries are required to build a SQL engine for BigQuery
!pip install --user SQLAlchemy==1.4.48 --quiet
!pip install --user sqlalchemy-bigquery --quiet
# Install pandas
! pip install --user --quiet pandas

# Install HuggingFace Datasets
! pip install --user --quiet datasets

# Install Python client for Google Search API
! pip install --user --quiet google-api-python-client

# PDF loader
! pip install --user --quiet pypdf

# Install similarity search library
# TO-DO: Replace with vertex AI Matching Engine
! pip install --user --quiet faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.17.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.1.4 which is incompatible.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.48 which is incompatible.


In [ ]:
PROJECT_ID = "iron-decorator-297513" # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
from google.cloud import aiplatform
from langchain.llms import VertexAI
import vertexai
aiplatform.init(project=PROJECT_ID, location=LOCATION)
vertexai.init(project=PROJECT_ID, location=LOCATION)
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
project_id = "iron-decorator-297513" # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
dataset_id = 'tg_Dataste' # @param {type:"string"}
table_name = 'LLMsales' # @param {type:"string"}
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import pandas as pd

In [ ]:
table_uri = f"bigquery://{project_id}/{dataset_id}"
engine = create_engine(f"bigquery://{project_id}/{dataset_id}")

In [ ]:
def bq_qna(question):
  #create SQLDatabase instance from BQ engine
  db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=[table_name])
  # db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=['bigquery-public-data.iowa_liquor_sales.sales'])

  #create SQL DB Chain with the initialized LLM and above SQLDB instance
  db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_intermediate_steps=True)


  _googlesql_prompt = """You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
  Use the following format:
  Question: "Question here"
  SQLQuery: "SQL Query to run"
  SQLResult: "Result of the SQLQuery"
  Answer: "Final answer here"
  Only use the following tables:
  {table_info}

  If someone asks for specific month, use ActivityDate between current month's start date and current month's end date

  Question: {input}"""

  BigQuerySQL_PROMPT = PromptTemplate(
      input_variables=["input", "table_info", "top_k"],
      template=_googlesql_prompt,
  )

  #passing question to the prompt template
  final_prompt = BigQuerySQL_PROMPT.format(input=question, table_info=table_name, top_k=10000)

  #pass final prompt to SQL Chain
  output = db_chain(final_prompt)


  return output['result'], output['intermediate_steps'][0]

In [ ]:
#i have installed the latest module of langchian which is stable for the creaet_SQL_Query chain
#please restart the cell after the installaiton is complete
!pip install langchain==0.0.353

  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.229
    Uninstalling langchain-0.0.229:
      Successfully uninstalled langchain-0.0.229


Using create_sql_query_chain to write queries and then using the db to run the query

In [ ]:
from langchain import SQLDatabase
from langchain.chains import sql_database
from google.cloud import aiplatform
from langchain.llms import VertexAI
import vertexai
PROJECT_ID = "iron-decorator-297513" # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
aiplatform.init(project=PROJECT_ID, location=LOCATION)
vertexai.init(project=PROJECT_ID, location=LOCATION)

table_name = ['Vehicle_ID_Table','topSpeed','Mileage']
table_uri = f"bigquery://{project_id}/{dataset_id}"
engine = create_engine(f"bigquery://{project_id}/{dataset_id}")

llm = VertexAI(
    model_name='text-bison@001',
    max_output_tokens=1024,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=table_name,sample_rows_in_table_info=10)
#chain = sql_database.query.create_sql_query_chain(llm,db,k=100,prompt=)




<ipython-input-6-d1200f07c031>:24: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=table_name,sample_rows_in_table_info=10)


In [ ]:
print(db.get_table_info())


CREATE TABLE `Mileage` (
	`vehicle_number` STRING, 
	`make` STRING, 
	`type` STRING, 
	`mileage` INT64
)

/*
10 rows from Mileage table:
vehicle_number	make	type	mileage
DEF456	Honda	SUV	45
MNOP	Skoda	SUV	52
678PQR	Acura	SUV	49
456DEF	Land Rover	SUV	85
890ABCD	Mini	SUV	46
PQR678	Nissan	Van	59
JKL012	Chevrolet	Coupe	34
234VWX	Mitsubishi	Sedan	35
TUVW	Peugeot	Sedan	27
ABC123	Toyota	Sedan	78
*/


CREATE TABLE `Vehicle_ID_Table` (
	`vehicle_id` INT64, 
	`vehicle_number` STRING, 
	`smoke_emission` FLOAT64, 
	`temperature` INT64
)

/*
10 rows from Vehicle_ID_Table table:
vehicle_id	vehicle_number	smoke_emission	temperature
2	DEF456	0.5	28
11	EFGH789	0.5	26
20	012JKL	0.5	32
28	456IJKL	0.5	23
1	ABC123	0.3	32
14	QRST678	0.3	30
22	678PQR	0.3	37
30	012QRST	0.3	28
8	VWX234	0.6	29
17	123ABC	0.6	21
*/


CREATE TABLE `topSpeed` (
	`vehicle_number` STRING, 
	`capacitor` INT64, 
	`cc` FLOAT64, 
	`top_speed` INT64
)

/*
10 rows from topSpeed table:
vehicle_number	capacitor	cc	top_speed
YZ123	1000	1.0	1

In [ ]:
print(db.table_info)


CREATE TABLE `Mileage` (
	`vehicle_number` STRING, 
	`make` STRING, 
	`type` STRING, 
	`mileage` INT64
)

/*
3 rows from Mileage table:
vehicle_number	make	type	mileage
None	None	None	None
None	None	None	None
None	None	None	None
*/


CREATE TABLE `Vehicle_ID_Table` (
	`vehicle_id` INT64, 
	`vehicle_number` STRING, 
	`smoke_emission` FLOAT64, 
	`temperature` INT64
)

/*
3 rows from Vehicle_ID_Table table:
vehicle_id	vehicle_number	smoke_emission	temperature
2	DEF456	0.5	28
11	EFGH789	0.5	26
20	012JKL	0.5	32
*/


CREATE TABLE `topSpeed` (
	`vehicle_number` STRING, 
	`capacitor` INT64, 
	`cc` FLOAT64, 
	`top_speed` INT64
)

/*
3 rows from topSpeed table:
vehicle_number	capacitor	cc	top_speed
YZ123	1000	1.0	100
123A	1300	1.0	100
567YZ1	1400	1.1	105
*/


In [ ]:
db.dialect

'bigquery'

In [ ]:
response = chain.invoke({"question": "How many records are there in all tables"})
print(response)

SELECT count(*) FROM Mileage


In [ ]:
response = chain.invoke({"question": "give me the vehilce id with maximum topspeed"})
print(response)

SELECT vehicle_id FROM topSpeed ORDER BY top_speed DESC LIMIT 1


In [ ]:
from langchain.prompts import PromptTemplate
template = """ Given an input question, first create a yntactically correct bigquery  to run, then look at the results of the query and return the answer.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL
Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Database Description:

There are 3 tables: Vehicle_ID, Mileage, and Topspeed.
Vehicle_ID contains: vehicle_id (unique numerical identifier), vehicle_number (alphanumeric code), smoke_emission (grams/km), and temperature (degrees Celsius).
Topspeed contains: vehicle_number, capacitor (Farads), cc (engine displacement), and top_speed (km/h).
Mileage contains: vehicle_number, make (vehicle manufacturer), type (vehicle model), and mileage (km/L).
The vehicle_id column is only present in the Vehicle_ID table.

Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

'Vehicle_ID_Table','topSpeed','Mileage'

Some examples of SQL queries that correspond to questions are:
"Find the average mileage for vehicles with a top speed greater than 200 km/h." : "SELECT AVG(mileage) AS average_mileage FROM Mileage JOIN Topspeed ON Mileage.vehicle_number = Topspeed.vehicle_number WHERE top_speed > 200;",
"List the vehicle model, make, and capacitor for vehicles with a smoke emission between 40 and 60 grams/km.": "SELECT type, make, capacitor FROM Mileage JOIN Vehicle_ID ON Mileage.vehicle_number = Vehicle_ID.vehicle_number WHERE smoke_emission BETWEEN 40 AND 60;",
"Retrieve the vehicle IDs and temperatures for vehicles with an engine displacement (cc) of 1500 or higher." : "SELECT vehicle_id, temperature FROM Vehicle_ID JOIN Topspeed ON Vehicle_ID.vehicle_number = Topspeed.vehicle_number WHERE cc >= 1500;",
"Calculate the total number of vehicles with a mileage of less than 15 km/L.":"SELECT COUNT(*) AS total_vehicles FROM Mileage WHERE mileage < 15;",
"Find the highest smoke emission value across all vehicles.":"SELECT MAX(smoke_emission) AS highest_smoke_emission FROM Vehicle_ID;",
"Show the vehicle model with the lowest temperature.":"SELECT type AS vehicle_model FROM Mileage JOIN Vehicle_ID ON Mileage.vehicle_number = Vehicle_ID.vehicle_number WHERE temperature = (SELECT MIN(temperature) FROM Vehicle_ID);"


Question: {input}"""


PROMPT = PromptTemplate(input_variables=["input"], template=template,)



question = "give me the vehicle id with maximum topspeed"

#final_prompt = PROMPT.format(input=question, top_k=10000,dialect='bigquery')


chain = sql_database.query.create_sql_query_chain(llm, db, k=100, prompt=PROMPT)
#chain = sql_database.query.create_sql_query_chain(llm,db,k=100,prompt=final_prompt)


#chain(final_prompt)








In [ ]:
response = chain.invoke({"question": "give me the vehicle id with maximum topspeed"})

In [ ]:
print(response)

SELECT vehicle_id FROM Topspeed ORDER BY top_speed DESC LIMIT 1;


In [ ]:
PROMPT

PromptTemplate(input_variables=['dialect', 'input', 'table_info', 'top_k'], template=' Given an input question, first create a yntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per BigQuery SQL\nBe careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n\nDatabase Description:\n\nThere are 3 tables: Vehicle_ID, Mileage, and Topspeed.\nVehicle_ID contains: vehicle_id (unique numerical identifier), vehicle_number (alphanumeric code), smoke_emission (grams/km), and temperature (degrees Celsius).\nTopspeed contains: vehicle_number, capacitor (Farads), cc (engine displacement), and top_speed (km/h).\nMileage contains: vehicle_number, make (vehicle manufacturer), type (vehicle model), and mileage (km/L).\nThe vehicle_id column is only present i

In [ ]:
print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [ ]:
print(final_prompt)

 Given an input question, first create a yntactically correct bigquery query to run, then look at the results of the query and return the answer.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL
Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Database Description:

There are 3 tables: Vehicle_ID, Mileage, and Topspeed.
Vehicle_ID contains: vehicle_id (unique numerical identifier), vehicle_number (alphanumeric code), smoke_emission (grams/km), and temperature (degrees Celsius).
Topspeed contains: vehicle_number, capacitor (Farads), cc (engine displacement), and top_speed (km/h).
Mileage contains: vehicle_number, make (vehicle manufacturer), type (vehicle model), and mileage (km/L).
The vehicle_id column is only present in the Vehicle_ID table.

Use the following format:

Question: "Question here"
SQLQuery: "SQL Query 

In [ ]:
table_name

['Vehicle_ID_Table', 'topSpeed', 'Mileage']

In [ ]:
prompt = {
    "database_description": "There are 3 tables: Vehicle_ID, Mileage, and Topspeed. To identify the vehicle_id, it needs to be joined with the vehicle_number column in other tables.\nVehicle_ID contains: vehicle_id (unique numerical identifier), vehicle_number (alphanumeric code), smoke_emission (grams/km), and temperature (degrees Celsius).\nTopspeed contains: vehicle_number, capacitor (Farads), cc (engine displacement), and top_speed (km/h).\nMileage contains: vehicle_number, make (vehicle manufacturer), type (vehicle model), and mileage (km/L).",
    "example_queries": [
        {
            "natural_language_query": "Find the average mileage for vehicles with a top speed greater than 200 km/h.",
            "sql_query": "SELECT AVG(mileage) AS average_mileage FROM Mileage JOIN Topspeed ON Mileage.vehicle_number = Topspeed.vehicle_number WHERE top_speed > 200;"
        },
        # ... other example queries
    ],
    "query": "Your natural language query here"
}


In [ ]:
Database Description:

There are 3 tables: Vehicle_ID, Mileage, and Topspeed.
Vehicle_ID contains: vehicle_id (unique numerical identifier), vehicle_number (alphanumeric code), smoke_emission (grams/km), and temperature (degrees Celsius).
Topspeed contains: vehicle_number, capacitor (Farads), cc (engine displacement), and top_speed (km/h).
Mileage contains: vehicle_number, make (vehicle manufacturer), type (vehicle model), and mileage (km/L).
The vehicle_id column is only present in the Vehicle_ID table.

Natural Language Query: "Find the average mileage for vehicles with a top speed greater than 200 km/h." SQL Query: ```sql SELECT AVG(mileage) AS average_mileage FROM Mileage JOIN Topspeed ON Mileage.vehicle_number = Topspeed.vehicle_number WHERE top_speed > 200;

Natural Language Query: "List the vehicle model, make, and capacitor for vehicles with a smoke emission between 40 and 60 grams/km." SQL Query: ```sql SELECT type, make, capacitor FROM Mileage JOIN Vehicle_ID ON Mileage.vehicle_number = Vehicle_ID.vehicle_number WHERE smoke_emission BETWEEN 40 AND 60;

Natural Language Query: "Retrieve the vehicle IDs and temperatures for vehicles with an engine displacement (cc) of 1500 or higher." SQL Query: ```sql SELECT vehicle_id, temperature FROM Vehicle_ID JOIN Topspeed ON Vehicle_ID.vehicle_number = Topspeed.vehicle_number WHERE cc >= 1500;

Natural Language Query: "Calculate the total number of vehicles with a mileage of less than 15 km/L." SQL Query: ```sql SELECT COUNT(*) AS total_vehicles FROM Mileage WHERE mileage < 15;

Natural Language Query: "Find the highest smoke emission value across all vehicles." SQL Query: ```sql SELECT MAX(smoke_emission) AS highest_smoke_emission FROM Vehicle_ID;

Natural Language Query: "Show the vehicle model with the lowest temperature." SQL Query: ```sql SELECT type AS vehicle_model FROM Mileage JOIN Vehicle_ID ON Mileage.vehicle_number = Vehicle_ID.vehicle_number WHERE temperature = (SELECT MIN(temperature) FROM Vehicle_ID);

